# Numerical validation of controllability of ansatze in Larocca et al., *Quantum* 6 824 (2022)

In [ ]:
import os
import logging
import time
import numpy as np
import h5py
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt

from fastdla.lie_closure import lie_closure
from fastdla.generators.hea import hea_generators
from fastdla.generators.spin_glass import spin_glass_generators
from fastdla.generators.heisenberg_hva import xxz_hva_generators
from fastdla.generators.tfim_hva import tfim_1d_hva_generators
from fastdla.generators.spin_chain import (magnetization_eigenspace, parity_eigenspace,
                                           translation_eigenspace, spin_flip_eigenspace)

logging.basicConfig(level=logging.WARNING)
logging.getLogger('fastdla').setLevel(logging.INFO)
LOG = logging.getLogger('fastdla')
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
jax.config.update('jax_enable_x64', True)

In [ ]:
data_file = ''
force_rewrite = False
representation = 'matrix'
max_workers = 110
algorithm = 'default'

In [ ]:
if data_file and not os.path.exists(data_file):
    with h5py.File(data_file, 'w'):
        pass

if representation == 'sparse':
    lc_kwargs = {'max_workers': max_workers}
    
    def get_generators(fn, *args, **kwargs):
        return fn(*args, **kwargs)

elif representation == 'matrix':
    lc_kwargs = {'algorithm': algorithm}

    def get_generators(fn, *args, **kwargs):
        return fn(*args, **kwargs).to_matrices()

### Hardware efficient ansatz

$$
\mathcal{G}_{\mathrm{HEA}} = \left\{ X_n, Y_n \right\}_{n=0}^{N_q-1} \cup
                                     \left\{ \sum_{n=0}^{N_q - 2} Z_n Z_{n+1} \right\}
$$

In [ ]:
run_calc = True
if data_file and not force_rewrite:
    with h5py.File(data_file, 'r') as source:
        if 'hea' in source:
            run_calc = False
            group = source['hea']
            nqs = group['num_qubits'][()]
            dla_dims = group['dla_dims'][()]

if run_calc:
    min_nq = 2
    max_nq = 6
    nqs = np.arange(min_nq, max_nq + 1)
    dla_dims = []
    runtimes = []

    # Compile with a small problem
    generators = get_generators(hea_generators, num_qubits=2)
    lie_closure(generators, **lc_kwargs)

    for num_qubits in nqs:
        LOG.info('num_qubits=%d', num_qubits)
        start = time.time()
        generators = get_generators(hea_generators, num_qubits=num_qubits)
        dla = lie_closure(generators, **lc_kwargs)
        runtime = time.time() - start
        LOG.info('DLA dimension %d (%.2f seconds)', dla.shape[0], runtime)
        dla_dims.append(dla.shape[0])
        runtimes.append(runtime)

    if data_file:
        with h5py.File(data_file, 'a') as out:
            if 'hea' in out:
                del out['hea']

            group = out.create_group('hea')
            group.create_dataset('num_qubits', data=nqs)
            group.create_dataset('dla_dims', data=dla_dims)
            group.create_dataset('runtimes', data=runtimes)

In [ ]:
x = np.linspace(nqs[0], nqs[-1], 100)
plt.plot(x, (2 ** x) ** 2 - 1, label='$(2^{N_q})^2-1$')
plt.scatter(nqs, dla_dims, label=r'$dim(\mathfrak{g})$')
plt.yscale('log')
plt.title('Hardware-efficient ansatz')
plt.xlabel('$N_q$', fontsize=16)
plt.xticks(nqs, labels=[f'{i}' for i in nqs])
plt.legend(fontsize=14);

### Spin glass



In [ ]:
run_calc = True
if data_file and not force_rewrite:
    with h5py.File(data_file, 'r') as source:
        if 'spin_glass' in source:
            run_calc = False
            group = source['spin_glass']
            nqs = group['num_qubits'][()]
            dla_dims = group['dla_dims'][()]

if run_calc:
    min_nq = 3
    max_nq = 6
    nqs = np.arange(min_nq, max_nq + 1)
    dla_dims = []
    runtimes = []

    # Compile with a small problem
    generators = get_generators(spin_glass_generators, num_qubits=3)
    lie_closure(generators, **lc_kwargs)

    for num_qubits in nqs:
        LOG.info('num_qubits=%d', num_qubits)
        start = time.time()
        generators = get_generators(spin_glass_generators, num_qubits=num_qubits)
        dla = lie_closure(generators, **lc_kwargs)
        runtime = time.time() - start
        LOG.info('DLA dimension %d (%.2f seconds)', dla.shape[0], runtime)
        dla_dims.append(dla.shape[0])
        runtimes.append(runtime)

    if data_file:
        with h5py.File(data_file, 'a') as out:
            if 'spin_glass' in out:
                del out['spin_glass']

            group = out.create_group('spin_glass')
            group.create_dataset('num_qubits', data=nqs)
            group.create_dataset('dla_dims', data=dla_dims)
            group.create_dataset('runtimes', data=runtimes)

In [ ]:
x = np.linspace(nqs[0], nqs[-1], 100)
plt.plot(x, (2 ** x) ** 2 - 1, label='$(2^{N_q})^2-1$')
plt.scatter(nqs, dla_dims, label=r'$dim(\mathfrak{g})$')
plt.yscale('log')
plt.title('Spin glass ansatz')
plt.xlabel('$N_q$', fontsize=16)
plt.xticks(nqs, labels=[f'{i}' for i in nqs])
plt.legend(fontsize=14);

### $XXZ$ model (subspace controllable)

The following code cell has no representation / algorithm switches.

In [ ]:
run_calc = True
if data_file and not force_rewrite:
    with h5py.File(data_file, 'r') as source:
        if 'xxz' in source:
            run_calc = False
            group = source['xxz']
            nspins = group['num_spins'][()]
            subspace_dims = group['subspace_dims'][()]
            dla_dims = group['dla_dims'][()]

if run_calc:
    min_nspin = 4
    max_nspin = 10
    nspins = np.arange(min_nspin, max_nspin + 1, 2)
    parity = 1
    subspace_dims = np.zeros((nspins.shape[0], max_nspin // 2 + 1), dtype=int)
    dla_dims = np.zeros((nspins.shape[0], max_nspin // 2 + 1), dtype=int)
    runtimes = np.zeros((nspins.shape[0], max_nspin // 2 + 1), dtype=float)

    for ispin, num_qubits in enumerate(nspins):
        LOG.info('num_spins=%d', num_qubits)
        generators = xxz_hva_generators(num_qubits).to_matrices()
        parity_subspace = parity_eigenspace(parity, num_spins=num_qubits, npmod=jnp)
        for imag, magnetization in enumerate(range(0, num_qubits + 1, 2)):
            LOG.info('magnetization=%d', magnetization)
            subspace = magnetization_eigenspace(magnetization, basis=parity_subspace, npmod=jnp)
            LOG.info('Subspace dimension %d', subspace.shape[1])
            generators_reduced = jnp.einsum('ij,gik,kl->gjl', subspace.conjugate(), generators,
                                            subspace)
            start = time.time()
            dla = lie_closure(generators_reduced, implementation=algorithm)
            runtime = time.time() - start
            LOG.info('DLA dimension %d (%.2f seconds)', dla.shape[0], runtime)
            subspace_dims[ispin, imag] = subspace.shape[1]
            dla_dims[ispin, imag] = dla.shape[0]
            runtimes[ispin, imag] = runtime

            break

    if data_file:
        with h5py.File(data_file, 'a') as out:
            if 'xxz' in out:
                del out['xxz']

            group = out.create_group('xxz')
            group.create_dataset('num_spins', data=nspins)
            group.create_dataset('subspace_dims', data=subspace_dims)
            group.create_dataset('dla_dims', data=dla_dims)
            group.create_dataset('runtimes', data=runtimes)

In [ ]:
from matplotlib.lines import Line2D

handles = [Line2D([0, 1], [0, 1], color='k'), Line2D([], [], color='k', linestyle='none', marker='o')]
labels = ['subspace $d^2$', r'$dim(\mathfrak{g})+1$']
for imag in range(nspins[-1] // 2 + 1):
    magnetization = imag * 2
    mask = nspins >= magnetization
    line, = plt.plot(nspins[mask], np.square(subspace_dims[mask][:, imag]))
    plt.scatter(nspins[mask], dla_dims[mask][:, imag] + 1)
    handles.append(line)
    labels.append(f'M={magnetization}')
plt.yscale('log')
plt.title('XXZ model HVA')
plt.xlabel('$N_q$', fontsize=16)
plt.xticks(nspins, labels=[f'{i}' for i in nspins])
plt.legend(handles, labels, fontsize=12);

### TFIM HVA

The following code cell has no representation / algorithm switches.

In [ ]:
run_calc = True
if data_file and not force_rewrite:
    with h5py.File(data_file, 'r') as source:
        if 'tfim' in source:
            run_calc = False
            group = source['tfim']
            nspins = group['num_spins'][()]
            dla_dims = group['dla_dims'][()]

if run_calc:
    min_nspin = 8
    max_nspin = 16
    nspins = np.arange(min_nspin, max_nspin + 1, 2)
    parity = 1
    z2_eigval = 1
    jphase = 0
    subspace_dims = np.zeros((nspins.shape[0], 2), dtype=int)
    dla_dims = np.zeros((nspins.shape[0], 2), dtype=int)
    runtimes = np.zeros((nspins.shape[0], 2), dtype=float)

    for ispin, num_qubits in enumerate(nspins):
        LOG.info('num_spins=%d', num_qubits)
        z2_subspace = spin_flip_eigenspace(z2_eigval, num_spins=num_qubits, npmod=jnp)
        for ibc, bc in enumerate(['open', 'periodic']):
            LOG.info('boundary condition: %s', bc)
            generators = tfim_1d_hva_generators(num_qubits, boundary_condition=bc).to_matrices()
            if bc == 'open':
                subspace = parity_eigenspace(parity, basis=z2_subspace, npmod=jnp)
            else:
                subspace = translation_eigenspace(jphase, basis=z2_subspace, npmod=jnp)
            LOG.info('Subspace dimension %d', subspace.shape[1])
            generators_reduced = jnp.einsum('ij,gik,kl->gjl', subspace.conjugate(), generators,
                                            subspace)
            start = time.time()
            dla = lie_closure(generators_reduced, implementation=algorithm)
            runtime = time.time() - start
            LOG.info('DLA dimension %d (%.2f seconds)', dla.shape[0], runtime)
            subspace_dims[ispin, ibc] = subspace.shape[1]
            dla_dims[ispin, ibc] = dla.shape[0]
            runtimes[ispin, ibc] = runtime

            break

    if data_file:
        with h5py.File(data_file, 'a') as out:
            if 'tfim' in out:
                del out['tfim']

            group = out.create_group('tfim')
            group.create_dataset('num_spins', data=nspins)
            group.create_dataset('dla_dims', data=dla_dims)
            group.create_dataset('runtimes', data=runtimes)

In [ ]:
for ibc, bc in enumerate(['open', 'periodic']):
    plt.plot(nspins, subspace_dims[:, ibc], label=f'subspace dim ({bc})')
    plt.scatter(nspins, dla_dims[:, ibc], label=fr'$dim(\mathfrak{{g}}_{{{bc}}})$')

x = np.linspace(nspins[0], nspins[-1], 200)
plt.plot(x, np.square(x), color='k', linestyle='--', label='$n^2$')
plt.plot(x, 1.5 * x, color='k', linestyle=':', label='3n/2')
plt.title('Transverse Ising model HVA')
plt.xlabel('$N_q$', fontsize=16)
plt.xticks(nspins, labels=[f'{i}' for i in nspins])
plt.legend(fontsize=12);